In [3]:
import os
import json
import datasets
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Math-1.5B")

In [22]:
ds = datasets.load_dataset("hendrycks/competition_math")
ds['train'][0]

{'problem': 'Let \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ if }x>2, \\\\\nx-5 &\\text{ if } -2 \\le x \\le 2, \\\\\n2x-b &\\text{ if } x <-2.\n\\end{array}\n\\right.\\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).',
 'level': 'Level 5',
 'type': 'Algebra',
 'solution': 'For the piecewise function to be continuous, the cases must "meet" at $2$ and $-2$. For example, $ax+3$ and $x-5$ must be equal when $x=2$. This implies $a(2)+3=2-5$, which we solve to get $2a=-6 \\Rightarrow a=-3$. Similarly, $x-5$ and $2x-b$ must be equal when $x=-2$. Substituting, we get $-2-5=2(-2)-b$, which implies $b=3$. So $a+b=-3+3=\\boxed{0}$.'}

In [10]:
ds['train'].filter(lambda x: any(i in x['level'] for i in ['3', '4', '5']))

Dataset({
    features: ['problem', 'level', 'type', 'solution'],
    num_rows: 5586
})

In [4]:
math500 = datasets.load_dataset("HuggingFaceH4/MATH-500")['test']
math500_qs = set(math500['problem'])

In [7]:
tmp = math500.filter(lambda x: x['level'] == 5)
tmp

Filter: 100%|██████████| 500/500 [00:00<00:00, 25792.37 examples/s]


Dataset({
    features: ['problem', 'solution', 'answer', 'subject', 'level', 'unique_id'],
    num_rows: 134
})

In [9]:
tmp[3]

{'problem': 'The proper divisors of 12 are 1, 2, 3, 4 and 6. A proper divisor of an integer $N$ is a positive divisor of $N$ that is less than $N$. What is the sum of the proper divisors of the sum of the proper divisors of 284?',
 'solution': 'Prime factorize $284=2^2\\cdot71$. The sum of the proper divisors of $284$ is\n\\begin{align*}\n1+2+2^2+71+2 \\cdot 71 &= (1+2+2^2)(1+71)-284 \\\\\n&= 220 \\\\\n&= 2^2\\cdot5\\cdot11.\n\\end{align*}Here we have used the observation that multiplying out $(1+2+2^2)(1+71)$ by distributing yields an expression which is the sum of all $6$ factors of $284.$ Applying this observation again, we find that the sum of the proper divisors of $220$ is $$(1+2+2^2)(1+5)(1+11)-220=7\\cdot 6\\cdot 12-220=\\boxed{284}.$$',
 'answer': '284',
 'subject': 'Number Theory',
 'level': 5,
 'unique_id': 'test/number_theory/737.json'}

In [24]:
tmp = ds['test'].filter(lambda x: x['problem'] not in math500_qs)
print(len(tmp))
tmp = tmp.shuffle(seed=42)
train2 = tmp.select(range(4000))
val = tmp.select(range(4000, 4500))

Filter: 100%|██████████| 5000/5000 [00:00<00:00, 105005.66 examples/s]

4500


In [25]:
train = datasets.concatenate_datasets([ds['train'], train2])
data = datasets.DatasetDict()
data['train'] = train
data['test'] = val
data

DatasetDict({
    train: Dataset({
        features: ['problem', 'level', 'type', 'solution'],
        num_rows: 11500
    })
    test: Dataset({
        features: ['problem', 'level', 'type', 'solution'],
        num_rows: 500
    })
})

In [26]:
data.save_to_disk('/dccstor/rag_data/math_data')

Saving the dataset (1/1 shards): 100%|██████████| 500/500 [00:00<00:00, 52838.30 examples/s]


In [24]:
res = json.load(open('1.5B_1.json'))

In [26]:
i = 4
print(res[i]['response'])
print(res[i]['answer'])
print(res[i]['problem'])

<think>
Okay, so I need to figure out how many zeros are at the end of 42!. Hmm, I remember that the number of zeros at the end of a factorial is related to how many times the number 10 is a factor in that factorial. Since 10 is 2 multiplied by 5, the number of zeros is determined by the number of pairs of 2s and 5s in the prime factorization of 42!.

But wait, I also recall that in factorial prime factorizations, there are usually more 2s than 5s. So, the number of zeros is actually determined by the number of times 5 is a factor in the numbers from 1 to 42. That means I need to count how many multiples of 5 are there in that range, and also account for higher powers like 25, 125, etc., because they contribute more than one 5.

Let me start by listing out the multiples of 5 between 1 and 42. So, 5, 10, 15, 20, 25, 30, 35, and 40. That seems like eight numbers. Each of these contributes at least one 5. So, that's 8 fives.

But some numbers contribute more than one 5. For example, 25 is